# Load libraries and data

In [4]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
# import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
import wandb

In [5]:
train_data = pd.read_csv(Path("..", "data", "processed", "train.csv"))
val_data = pd.read_csv(Path("..", "data", "processed", "val.csv"))

# Parameters & wandb

In [6]:
EMBEDDING_DIM = 100
MAX_LEN = 150
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}")

Using cuda


In [7]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: daniele-didino to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Tokenizer

In [8]:
# Tokenizer (basic word splitting)
def basic_tokenizer(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # Remove special characters
    return text.split()


def build_vocab(texts):
    token_counts = Counter()
    for text in texts:
        cleaned_text = basic_tokenizer(text)
        token_counts.update(cleaned_text)
    vocab = {word: idx + 2 for idx, (word, _) in enumerate(token_counts.most_common())}  # Reserve index 0 for padding, 1 for unknown
    vocab["<PAD>"] = 0
    vocab["<UNK>"] = 1
    return vocab


# Convert texts to numerical sequences
def encode_text(text, vocab, max_len=150):
    tokens = basic_tokenizer(text)
    encoded = [vocab.get(word, vocab["<UNK>"]) for word in tokens[:max_len]]
    return np.pad(encoded, (0, max_len - len(encoded)), constant_values=vocab["<PAD>"])[:max_len]

# Model

In [9]:
# Dataset Class
class ToxicDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=150):
        texts = [encode_text(text, vocab, max_len) for text in texts]
        self.texts = [torch.tensor(text, dtype=torch.long) for text in texts]
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "input_ids": self.texts[idx],
            "labels": self.labels[idx]
        }


# Model: CNN + GRU
class ToxicClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_filters, kernel_size, dropout, num_classes):
        super().__init__()
        # Embedding layers
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # CNN layer
        self.conv = nn.Conv1d(
            in_channels=embed_dim,
            out_channels=num_filters,
            kernel_size=kernel_size,
            padding=1)
        self.pool = nn.AdaptiveMaxPool1d(50) # This reduces the sequence length
        # GRU layer
        self.gru = nn.GRU(
            input_size=num_filters,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True)
        # Fully connected layer
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        x = self.embedding(x) # (batch_size, seq_len, embed_dim)
        x = x.permute(0, 2, 1) # change shape for conv1d (batch_size, channels, seq_len)
        x = torch.relu(self.conv(x))
        x = self.pool(x)
        x = x.permute(0, 2, 1) # change shape back for GRU (batch_size, seq_len, channels)
        x, _ = self.gru(x)
        x = self.dropout(x[:, -1, :]) # take the last time step
        return self.fc(x)


# Training function
def model_train(model, train_loader, val_loader, criterion, optimizer, epochs, device):
    model.to(device)

    for epoch in range(epochs):
        model.train()  # set model to training mode
        total_train_loss = 0
        all_preds = []
        all_labels = []

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            outputs = torch.sigmoid(outputs)
            all_preds.append(outputs.cpu().detach().numpy())
            all_labels.append(labels.cpu().numpy())
        
        # Compute Loss
        train_loss = total_train_loss / len(train_loader)

        # Compute AUC_ROC
        all_preds = np.concatenate(all_preds, axis=0).T
        all_labels = np.concatenate(all_labels, axis=0).T
        train_roc_auc = np.mean(
            [roc_auc_score(y_true, y_pred) for y_true, y_pred in zip(all_labels, all_preds)]
        )

        # Validation Step
        model.eval()
        val_preds = []
        val_labels = []
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids)
                
                # Compute validation loss
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()

                val_preds.append(outputs.cpu().numpy())
                val_labels.append(labels.cpu().numpy())
        
        # Compute validation loss
        val_loss = total_val_loss / len(val_loader)

        # Compute AUC_ROC
        val_preds = np.concatenate(val_preds, axis=0).T
        val_labels = np.concatenate(val_labels, axis=0).T
        val_roc_auc = np.mean(
            [roc_auc_score(y_true, y_pred) for y_true, y_pred in zip(val_labels, val_preds)]
        )

        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f} | AUC_ROC: {train_roc_auc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | AUC_ROC: {val_roc_auc:.4f}")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_roc_auc": train_roc_auc,
            "val_loss": val_loss,
            "val_roc_auc": val_roc_auc,
        })

    return train_loss, train_roc_auc, val_loss, val_roc_auc


# wandb Sweeps

In [10]:
train_input = train_data.comment_text.to_list()
train_labels = train_data.loc[:, ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

val_input = val_data.comment_text.to_list()
val_labels = val_data.loc[:,  ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

vocab = build_vocab(train_input)

# Prepare train dataset
train_dataset = ToxicDataset(train_input, train_labels, vocab, MAX_LEN)

# Prepare validation dataset
val_dataset = ToxicDataset(val_input, val_labels, vocab, MAX_LEN)

In [11]:
sweep_config = {
    "method": "bayes", # "random" or "grid" or "bayes"
    "metric": {"name": "val_roc_auc", "goal": "maximize"},
    "parameters": {
        "vocab_size": {"values": [len(vocab)]},
        "embed_dim": {"values": [50, 100, 200]},
        "hidden_dim": {"values": [64, 128, 256]},
        "kernel_size": {"min": 2, "max": 5},
        "num_filters": {"values": [64, 128, 256]},
        "dropout": {"min": 0.2, "max": 0.5},
        "learning_rate": {"min": 1e-4, "max": 1e-2, "distribution": "log_uniform_values"},
        "batch_size": {"values": [16, 32, 64]},
        "epochs": {"min": 1, "max": 5},
    }
}

# Create the sweep
sweep_id = wandb.sweep(sweep_config, project="toxic_comment_clf")

Create sweep with ID: bosycdpn
Sweep URL: https://wandb.ai/daniele-didino/toxic_comment_clf/sweeps/bosycdpn


In [ ]:
# Define the training function
def train_sweep():

    # EPOCHS = 2
    num_classes = 6  # toxic, severe_toxic, obscene, threat, insult, identity_hate

    with wandb.init() as run:
        config = wandb.config # sample hyperparameters
        
        # Initialize DataLoaders
        train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=True)
    
        # Initialize model
        model = ToxicClassifier(
            config["vocab_size"],
            config["embed_dim"],
            config["hidden_dim"],
            config["num_filters"],
            config["kernel_size"],
            config["dropout"],
            num_classes=num_classes)
        model.to(DEVICE)
    
        # Loss
        criterion = nn.BCEWithLogitsLoss() # Multi-label loss
    
        # Optimizer
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    
        # Training loop
        _ = model_train(model, train_dataloader, val_dataloader, criterion, optimizer, config["epochs"], DEVICE)

In [ ]:
# Launch the sweep
wandb.agent(sweep_id, function=train_sweep, count=20)